#### Package imports

In [5]:
!jupyter nbextension enable --py widgetsnbextension
#interactive web map package
import folium 
from folium import features
import shapefile
from json import dumps
from folium import plugins

#Miscellanous
from IPython.core.display import display #display inline package
from ipywidgets import interact, interactive, fixed, FloatProgress
import ipywidgets as widgets
import pdb #debugging tool
import pandas as pd # pandas dataframe package
import pandas
from datetime import datetime #date conversion tool
from xlrd.xldate import xldate_as_tuple #xldate converter
from urllib2 import urlopen #get data from web tool
#from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image
import utm #lat long to utms
import zipfile #unzip tool
from IPython.display import clear_output
from scipy.signal import correlate #rainfall correlation

#numpy packages and tools
import numpy as np
from numpy import linspace
from numpy.fft import fft, ifft, fft2, ifft2, fftshift

#Spatial data packages
import rasterio #io raster data 
#from osgeo import gdal

#Plotly tools
import plotly.tools as tls
tls.set_credentials_file(username='NTPlotly', api_key='48kd2al3f2') #my plotly credentials (please dont use they cost me $$)
import plotly.plotly as py
#from plotly.graph_objs import *
import plotly.graph_objs as go
from plotly import tools
import cufflinks as cf
import matplotlib

#scipy packages
from scipy.misc import derivative #derivates of functions tool
from scipy.interpolate import UnivariateSpline #basic spline fitting tool
from scipy.signal import gaussian #gaussian filtering tool
from scipy.ndimage import filters #anotehr filtering tool
from scipy import stats #core scipy stats package
from scipy.integrate import simps #simpsons rule package for trapezoid calcualtions in area under curve calcs

#Basic math tools
from math import log
from math import factorial
import random

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


#### NPV Peak Detection Assessment

An assessment of the npv peak detection method to identify major growth peaks using a classification of the Standard Precipitation Index. The following catergories have been used to assess the proportions of peaks in each class as a measure of the accuracy of the peak detection method to identify peaks associated with significat rainfall periods.

|SPI Start | SPI End | Catergory|
|:--------:|:-------:|:---------:|
| 2        |+2       |Extremely wet|
|1.5       |1.99     |Very wet|
|1.0       |1.49     |Moderately wet|
|-0.99     |0.99     |Near normal|
|-1.0      |-1.49    |Moderately dry|
|-1.5      |-1.99    |Severely dry|
|-2        |<-2      | Extremely dry |


In [6]:
#Enter siteId from map exactly as it is displayed ie. 'hmd20' or 'bsp14a'
text = widgets.Text(description="Site Id from map")
display(text)

Widget Javascript not detected.  It may not be installed or enabled properly.


#### Standardised Precipitation Index Time series

In [7]:
#Function to download a time series of each field sites Standardised Precipitation Index (SPI) values. SPI has been calculated for a 20 month window for each 'standard' season for a time series between 1981 - 2014.
def getSpiData(site,period):
            
    url = "https://dl.dropboxusercontent.com/u/53285700/csvs/spiOutput" + period + ".csv" #adds the chosen period to the url NOTE: 36 months appears to be the most representative of an arid climate
    
    u = urlopen(url)
    data = u.read()
    u.close()
    filename = "data/spi.csv"
 
    with open(filename, "wb") as f :
        f.write(data)
        
    df = pandas.read_csv(filename,header =0,parse_dates=['season'],dayfirst=True)
    
    mask = df['siteId']==site
    
    newDf = df[mask]
    
    return newDf 

In [8]:
def getOutputData(filename,pickleName,x,y):
    
    df = pd.read_csv(filename,header=0)
    
    df2 = pd.read_pickle(pickleName)
    
    #pdb.set_trace()
    
    modelledData = df.loc[(df["x"] == x) & (df["y"] == y)]
    
    dfV = df2.loc[(df2["x"] == x) & (df2["y"] == y)]
    
    dfV['dates'] = pandas.to_datetime(dfV['dates'])
    
    modelledData['sDate'] = pandas.to_datetime(modelledData['sDate'])
    
    modelledData['eDate'] = pandas.to_datetime(modelledData['eDate'])
    
    
    
    return dfV, modelledData

In [9]:
#simple function to find the closest date in a list of dates, i.e. find the closest image date to a monthly rainfall date
def nearestDate(base, dates):
    
    #pdb.set_trace()
        
    nearness = { abs(base - date) : date for date in dates }
    
    return nearness[min(nearness.keys())]

In [10]:
# function to apply a cross correlation to two datsets to determine the positive or negative shift required to align one data set to the other, uses a fast fourier transform.

def cross_correlation_using_fft(x, y):

    f1 = fft(x)
    
    f2 = fft(np.flipud(y))
    
    cc = np.real(ifft(f1 * f2))
    
    return fftshift(cc)
 

In [11]:
# function to compute the shift between the two datasets (npv and SPI)

def compute_shift(x, y):
    
    assert len(x) == len(y)
    
    c = cross_correlation_using_fft(x, y) # calls the fft function to determine the lag between the two data sets
    
    assert len(c) == len(x)
    
    zero_index = int(len(x) / 2) - 1
    
    shift = zero_index - np.argmax(c)
    
    return shift

In [12]:
#Function that takes both the input pixel spi and npv data and returns arrays of adjusted spi values and dates at npv peak locations.

def doPeakAssessment(dfV,modelledData,spiSiteData):
      
    #find the peaks and troughs
    
    peakMask = dfV['peaks'] == True

    peaks = dfV[peakMask]

    troughMask = dfV['troughs'] == True

    troughs = dfV[troughMask]
    
    #check the start and ends
    pS= peaks['dates'].iloc[0]-troughs['dates'].iloc[0]

    if pS.days >0:

        peakStart = False

    else:
        peakStart = True

    pE= peaks['dates'].iloc[-1]-troughs['dates'].iloc[-1]

    if pE.days >0:

        peakEnd = True
    else:
        peakEnd = False

    #Four possible scenarios

    #Scenario 1 is starts with trough and ends with trough

    troughInd = troughs.index.tolist()

    peaksInd = peaks.index.tolist()

    if peakStart == False and peakEnd == False:

        sliceIndex = troughInd

    #scenario 2 is starts with peak and ends with peak

    if peakStart == True and peakEnd == True:       

        sliceIndex = troughInd

        sliceIndex.insert(0, peaksInd[0]) # insert peak start index

        sliceIndex.insert(len(sliceIndex), peaksInd[-1]) #inser peak end index

    #scenario 3 is starts with peak and ends with trough

    if peakStart == True and peakEnd == False:       

        sliceIndex = troughInd

        sliceIndex.insert(0, peaksInd[0]) # insert peak start index

    #scenario 3 is starts with peak and ends with trough

    if peakStart == False and peakEnd == True:       

        sliceIndex = troughInd

        sliceIndex.insert(len(sliceIndex), peaksInd[-1]) # insert peak start index

    shifted = []

    cycleLength = []

    noYears = []

    #loop through and slice the data into growth cycles of NPV, rainfall and spi
    for i in range(len(sliceIndex)):

        if i <len(sliceIndex)-1:
            
          
            #slice NPV

            start = sliceIndex[i]

            end = sliceIndex[i+1]

            cycle = dfV['npv'].iloc[start:end].tolist()
            
            cycleDates = dfV['dates'].iloc[start:end]
            
            #loop to get spi values at cycle dates
               
            spiSliced = []
            
            for z in cycleDates:
                
                d = nearestDate(z, spiSiteData['season'])
                
                mask = spiSiteData['season']== d

                spiInd = mask[mask == True].index.tolist()
                
                spiSliced.append(spiSiteData['stats'].loc[spiInd[0]])
            

            #slice spi
            startDate = dfV['dates'].iloc[start]

            endDate = dfV['dates'].iloc[end]

            matchS = nearestDate(startDate, spiSiteData['season'])

            matchE = nearestDate(endDate, spiSiteData['season'])

            dateDif = matchS - matchE

            dateDif = abs(dateDif.days)

            years = dateDif/365

            time = np.arange(1-len(cycle),len(cycle))
         
          
            shift = compute_shift(cycle, spiSliced)
            
            shifted.append(int(shift))

            cycleLength.append(len(cycle))

            noYears.append(years)   
                      
    abShifted = abs(np.array(shifted)) #convert to absolute values for testing

    lengthCompare = abShifted > cycleLength #this is a test to check if the cross correlation has resulted in an error, i.e the lag is greater than the length of the cycle.

    noYearsTest = np.array(noYears) > 3 #this is a test to find cycles with a length greater than three years, only peaks with a cycle greater than 3 years will be shifted.

    peaksArray = np.array(peaksInd) #convert the original peakInd to an array
    
    peaksLength = peaksArray[lengthCompare] #return indexs of the peaks that have a greater cycle length than the calculated shift, i.e. lag error
    
    peaksYears = peaksArray[noYearsTest] #returns the indexes of peaks that have a cycle length greater than 3 years. These are the ones that we can apply the lag shift too.

    shiftedArray = np.array(shifted) #convert to np array
    
    shiftedYears = shiftedArray[noYearsTest] #returns the lag for only the peaks > 3 years

    #Loop to go through and replace shiftedYears values with a zero that have been filtered on years but in the rare case have a lag error.
    for i in peaksLength:

        comp = np.where(i== peaksYears) #search peaks years for a peaks length value and return its index

        comp = np.array(comp) #convert to a np array


        if comp.size:

            comp = comp[0] 

            shiftedYears[comp] = 0 #replace the error with a zero
        
    #Do the adjustment for lag
    
    adjusted = peaksYears+shiftedYears
    
    adjusted4Plot = peaksArray + shiftedArray
    
    for n in range(len(adjusted4Plot)):
        
        if adjusted4Plot[n] > len(dfV)-1:
            
            adjusted4Plot[n]=len(dfV)-1
                        
        if adjusted4Plot[n] < 0:
                   
            adjusted4Plot[n] = 0
   
    #loop to get the spi values from the adjusted index's

    adjustedDates = dfV['dates'].iloc[adjusted]
    
    adjustedDates4Plot = dfV['dates'].iloc[adjusted4Plot]

    adjustedSpi = []
    
    adjustedSpiDate = []
    
    adjustedSpi4Plot = []
    
    adjustedSpiDate4Plot = []
    
    for i in adjustedDates:

        match = nearestDate(i, spiSiteData['season'])

        mask = spiSiteData['season']== match

        spi = spiSiteData[mask]

        adjustedSpi.append(spi['stats'].iloc[0])
        
        adjustedSpiDate.append(spi['season'].iloc[0])
        
    for i in adjustedDates4Plot:

        match = nearestDate(i, spiSiteData['season'])

        mask = spiSiteData['season']== match

        spi = spiSiteData[mask]

        adjustedSpi4Plot.append(spi['stats'].iloc[0])
        
        adjustedSpiDate4Plot.append(spi['season'].iloc[0])
    
    #loop to get the spi values from the peak index without the lag adjustment 

    noNadjustedDates = dfV['dates'].iloc[peaksInd]  
      
    noNadjustedSpi = []

    noNadjustedSpiDate = []
    
    for i in noNadjustedDates:

        match = nearestDate(i, spiSiteData['season'])

        mask = spiSiteData['season']== match

        spi = spiSiteData[mask]

        noNadjustedSpi.append(spi['stats'].iloc[0])
        
        noNadjustedSpiDate.append(spi['season'].iloc[0])

    return adjustedSpi,adjustedSpiDate,noNadjustedSpi,noNadjustedSpiDate,adjustedSpi4Plot,adjustedSpiDate4Plot   

In [13]:
# function to process all of the pixels or at least a subset of the 2km x 2km image chip

def doDoAllSpiAssess(period):
    
    #get data and arrange
             
    spiSiteData = getSpiData(text.value,period) # get the spi site data, this is as at the site coord, havent averaged anything across the chip i.e. 20 x 20 due to the course nature of the rainfall grids.
    
    #npv,profile = openTif('data/chip.tif') #get npv data
    
    #(nBands,nRows,nCols) = npv.shape
    
    #temp = pandas.read_csv('data/dates.csv',header =0,parse_dates=['0'],dayfirst=True) #reads the csv into a pandas array
    
    #dates = pandas.DataFrame()
    
    #dates['dates']=temp['0']
    
    # Loop through the image chip and do the peak assessment
         
    allAdjustedSpi = []

    allNoNadjustedSpi = []
    
    allAdjustedSpi4Plot = []
    
    f = FloatProgress(min=0, max=67/3)
    
    display(f)#progress bar

    for i in range(67/3): # note: I have divided by three to reduce the number of pixels i.e. a subset of the image chip, this is to reduce the processing time
                        
        print str(i+1) + " of " +str(67/3)
        
        f.value = i
                
        ff = FloatProgress(min=0, max=67/3)
    
        display(ff)#progress bar
        
        for ii in range(67/3): 
            
            ff.value = ii     
                       
            dfV,modelledData = getOutputData("data/output.csv",'data/outputTimeSeriesPk.pkl',int(i),int(ii))
            
            adjustedSpi,adjustedSpiDate,noNadjustedSpi,noNadjustedSpiDate,adjustedSpi4Plot,adjustedSpiDate4Plot  = doPeakAssessment(dfV,modelledData,spiSiteData)
            
            for j in adjustedSpi:

                allAdjustedSpi.append(j)

            for k in noNadjustedSpi:

                allNoNadjustedSpi.append(k)
                
            for l in adjustedSpi4Plot:
                
                allAdjustedSpi4Plot.append(l)
                
    return allAdjustedSpi, allNoNadjustedSpi, allAdjustedSpi4Plot

In [14]:
# function to loop through a number of defined spi periods and calculate the shift, used namely in the testing phase, have hard wired to 36 months for the arid study area

def doSpiProcess(btn):
            
    print 'Processing started.....'
    
    print "Note: this can take up to 20 minutes"
            
    #periods = ["6m","12m","24m","36m"] # this could be used to select other periods, i.e. for more seasonally driven sytems
    
    periods = ["36m"]
            
    results = pd.DataFrame()

    for i in range(len(periods)):           

            allAdjustedSpi, allNoNadjustedSpi, allAdjustedSpi4Plot  = doDoAllSpiAssess(periods[i])

            tempDF = pd.DataFrame(allAdjustedSpi,columns=['adjusted_'+periods[i]])

            results = pd.concat([results,tempDF])

            tempDF = pd.DataFrame(allNoNadjustedSpi,columns=['raw_'+periods[i]])

            results = pd.concat([results,tempDF])
            
            tempDF = pd.DataFrame(allAdjustedSpi4Plot,columns=['adjusted4Plot_'+periods[i]])

            results = pd.concat([results,tempDF])

    cols =  results.columns 
    
    results.to_csv("data/spiPeakResult.csv") #write to output csv
    
    #next stage is classify the spi data into the following wet / dry catergories
    
    '''
    Based on Shah etal 2015 'Drought Index Computation Using Standardized Precipitation Index (SPI) Method For Surat District, Gujarat'  SPI catergories:
    
        SPI Range Category
    + 2 to more Extremely wet
    1.5 to 1.99 Very wet
    1.0 to 1.49 Moderately wet
    -0.99 to 0.99 Near normal
    -1.0 to -1.49 Moderately dry
    -1.5 to -1.99 Severely dry
    -2 to less Extremely dry    
    '''
    spiInd = [-2,-2,-1.5,-1,1.0,1.5,2.0]
       
    allCats = []
    
    # loop through spi values and catergorise

    for i in cols:

        m = results[i]

        notNulls = pd.notnull(m)

        sizeM = sum(notNulls)

        cats = []

        for ii in range(len(spiInd)):

            if ii == 0:

                sliced = m[notNulls]

                testA = sliced<spiInd[ii]

                resultA = sliced[testA]

                resSum = len(resultA)

                perc = float(resSum)/float(sizeM)*100

                cats.append(perc)

            elif ii == len(spiInd)-1:

                sliced = m[notNulls]

                testA = sliced>spiInd[ii]

                resultA = sliced[testA]

                resSum = len(resultA)

                perc = float(resSum)/float(sizeM)*100

                cats.append(perc)

            else:            

                sliced = m[notNulls]

                testA = sliced>spiInd[ii]

                resultA = sliced[testA]

                testB = resultA<spiInd[ii+1]

                resultB = resultA[testB]

                resSum = len(resultB)

                perc = float(resSum)/float(sizeM)*100

                cats.append(perc)

        allCats.append(cats) 

    resultAllCats = pd.DataFrame()

    for i in range(len(allCats)):

        resultAllCats[cols[i]]=allCats[i]
        
    print 'Processing complete'

    resultAllCats.to_csv("data/spiPeakAssessment.csv") #write to output csv
    
    

In [11]:
btn = widgets.Button(description="Evalute peak detection!")
btn.on_click(doSpiProcess)
display(btn)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [15]:
doSpiProcess('a')

Processing started.....
Note: this can take up to 20 minutes


Widget Javascript not detected.  It may not be installed or enabled properly.


1 of 22


Widget Javascript not detected.  It may not be installed or enabled properly.


ValueError: min() arg is an empty sequence

#### Plot the results

Creates a BAR plot of the proportion of peaks detecting in each spi catergory for both lag adjusted spi, non-lag adjusted spi for both major and all growth cycles (major >3 years)

In [23]:
def dospiAssessPlot(b):

    temp = pandas.read_csv('data/spiPeakAssessmentAllSites.csv',header =0) #import the output from the assessment (see methods section)
    
    columns = temp.keys()
    
    #pdb.set_trace()
    
    columns = columns[1:3]
    
    temp = temp[columns]
    
    #Grouping of Shah etal 2015 catergories
    
    normToWet = temp.iloc[3:] # Normal -1 to 1 to extremely wet +2

    normToWetSums = []
        
    cols = temp.columns
    
    cols = ['All years','Long term (>3 years)']
    
    # loop through and create bar chart series for each catergory of the proportions of peak detections.
         
    data = []  
    
    names = ['Extremely dry','Severely dry','Moderately dry','Near normal','Moderately wet','Very wet','Extremely wet']
    
    for i in range(len(temp)):
    
        x = cols
        
        y = temp.iloc[i]
        
        r = 255-i*30
        
        g = 255-i*30
        
        b = 255-i*30
        
        colour = "rgb(" + str(r)+','+str(g)+','+str(b)+')'
        
        trace = go.Bar(x=x,y=y,name=names[i],marker=dict(color=colour))
    
        data.append(trace)
    
    
    layout = go.Layout(barmode='group',title='Proportions of peaks detected in each Standardised Precipitation Index (SPI) Catergory',xaxis=dict(title='Standardised Precipitation Index Catergories'),yaxis=dict(title='Proportion %'))
    
    fig = go.Figure(data=data, layout=layout)
    
    spiPlot = py.iplot(fig, filename='grouped-bar')
    
    display(spiPlot)


In [13]:
btn = widgets.Button(description="Display peak assessment plot!")
btn.on_click(dospiAssessPlot)
display(btn)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [24]:
dospiAssessPlot('b')

Create a plot of both NPV and SPI with associated spi peak adjusted points at both all peaks and only peask with a growth period greater than three years

In [0]:
def doDeficitPlot(peaks,troughs,data,spiSiteData,adjustedSpi,adjustedSpiDates,adjustedSpi4plot,adjustedSpiDates4Plot):
    
    #pdb.set_trace()
    
    peakSpi = []
    
    peakSpiDates = []
    
    troughSpi = []
    
    troughSpiDates = []
    
    for i in peaks['dates']:

        match = nearestDate(i, spiSiteData['season'])

        mask = spiSiteData['season']== match

        spi = spiSiteData[mask]

        peakSpi.append(spi['stats'].iloc[0])

        peakSpiDates.append(spi['season'].iloc[0])

    for i in troughs['dates']:

        match = nearestDate(i, spiSiteData['season'])

        mask = spiSiteData['season']== match

        spi = spiSiteData[mask]

        troughSpi.append(spi['stats'].iloc[0])

        troughSpiDates.append(spi['season'].iloc[0])
    
    
    spiAdjPeak4Plot = go.Scatter(x=adjustedSpiDates4Plot,y=adjustedSpi4plot,name='SPI-Peak All (lag)',mode = 'markers', marker = dict(color = ('rgb(255, 255, 0)'),size = 14,symbol='circle'),yaxis='y2')

    data.append(spiAdjPeak4Plot)
    
    spiAdjPeak = go.Scatter(x=adjustedSpiDates,y=adjustedSpi,name='SPI-Peak (lag)',mode = 'markers', marker = dict(color = ('rgb(255, 0, 0)'),size = 14,symbol='circle'),yaxis='y2')

    data.append(spiAdjPeak)
    
    
    spiPeak = go.Scatter(x=peakSpiDates,y=peakSpi,name='SPI-Peak',mode = 'markers', marker = dict(color = ('rgb(0, 0, 255)'),size = 8,symbol='circle'),yaxis='y2')

    data.append(spiPeak)
   

    allSpi = go.Scatter(x=spiSiteData['season'],y=spiSiteData['stats'],name='SPI',mode = 'lines', marker = dict(color = ('rgb(0, 0, 255)'),size = 8,symbol='circle'),yaxis='y2')

    data.append(allSpi)

    layout = go.Layout(yaxis=dict(title='Non-Photosynthetic Vegetation Cover %'),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=14, color = ('rgb(1, 1, 1)')),yaxis2=dict(title='Standardised Precipitation Index',titlefont=dict(color='rgb(148, 103, 189)'),tickfont=dict(color='rgb(148, 103, 189)'),overlaying='y',side='right'))

    fig = go.Figure(data=data,layout=layout)

    filename1 = "researchProject/spi"

    plot = py.iplot(fig, filename=filename1)       
    
    return  plot
    

In [0]:
#Function that gets both the npv and spi data once the button is clicked

def on_button_clicked_spi(b):
           
    site = text.value
  
    temp1 = temp2.value
    
    spiSiteData = getSpiData(site,temp1)
    
    r = row.value
    
    c = col.value
    
    temp = pandas.read_csv('data/dates.csv',header =0,parse_dates=['0'],dayfirst=True) #reads the csv into a pandas array
    
    dates = pandas.DataFrame()
    
    dates['dates']=temp['0']
    
    dfV,modelledData = getOutputData("data/output.csv",'data/outputTimeSeriesPk.pkl',int(r),int(c))

    peakMask = dfV['peaks'] == True

    peaks = dfV[peakMask]

    troughMask = dfV['troughs'] == True

    troughs = dfV[troughMask]

    locFig,data,layout = doLocationPlots(dfV,modelledData)
    
    #pdb.set_trace()

    adjustedSpi,adjustedSpiDates,noNadjustedSpi,noNadjustedSpiDates,adjustedSpi4Plot,adjustedSpiDates4Plot = doPeakAssessment(dfV,modelledData,spiSiteData)
    
    st = dates.iloc[0]
    
    st = st.iloc[0]
    
    match = nearestDate(st, spiSiteData['season'])

    mask = spiSiteData['season']>= match

    spiSiteData1 = spiSiteData[mask]

    plot = doDeficitPlot(peaks,troughs,data,spiSiteData1,adjustedSpi,adjustedSpiDates,adjustedSpi4Plot,adjustedSpiDates4Plot)  
        
    display(plot)

In [0]:
def doLocationPlots(df,modelledData):
     
    
    newDf = df.set_index(['dates'])
    
    sDateSliced = newDf.ix[modelledData['sDate']]
        
    eDateSliced = newDf.ix[modelledData['eDate']] 
   
        
    trace1=[]
    
    trace1a = go.Scatter(x=df['dates'],y=df['fittedSpline']-100.0,name='NPV Spline',mode = 'lines', line = dict(color = ('rgb(1, 1, 1)'),width = 2))
    
    trace1.append(trace1a)
    
    trace1b = go.Scatter(x=modelledData['sDate'],y=sDateSliced['fittedSpline']-100.0,name='Peaks',mode = 'markers', marker = dict(color = ('rgb(1, 1, 1)'),size = 13,symbol='circle'))
    
    trace1.append(trace1b)
    
    trace1c = go.Scatter(x=modelledData['eDate'],y=eDateSliced['fittedSpline']-100.0,name='Troughs',mode = 'markers', marker = dict(color = ('rgb(1, 1, 1)'),size = 13,symbol='triangle-up'))
   
    trace1.append(trace1c)
    
    trace1d = go.Scatter(x=df['dates'],y=df['npv']-100.0,name='NPV',mode = 'markers', marker = dict(color = ('rgb(1, 1, 1)'),size = 1,symbol='circle'))
    
    trace1.append(trace1d)
            
    layout = go.Layout(yaxis=dict(title='Non-Photosynthetic Vegetation Cover %'),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=14, color = ('rgb(1, 1, 1)')))
      
    fig = go.Figure(data=trace1, layout=layout)
        
    #peakUrl = py.iplot(fig,filename=filename)   
    
    #pdb.set_trace()
    
    return fig,trace1,layout

In [0]:
row = widgets.Text(description="y:(0-67)")
col = widgets.Text(description="x:(0-67)")
row.value = '33'
col.value = '33'
display(row)
display(col)

In [0]:
temp2 = widgets.Dropdown(options ={'6 months':'6m','12 months':'12m','24 months':'24m','36 months':'36m'}) # default is 36 months

In [0]:
display(temp2)

In [0]:
button = widgets.Button(description="Rainfall deficit")
display(button)
button.on_click(on_button_clicked_spi)

### NPV Ground Cover - Peak Detection Assessment